Load sparkmagic before start


In [1]:
%load_ext sparkmagic.magics

## Azure ML Setup

In [2]:
from azureml.core import Workspace, Dataset

In [3]:
ws = Workspace.from_config()

In [4]:
dsetdata = 'noaa-isd-files'
data_url = 'https://azureopendatastorage.blob.core.windows.net/isdweatherdatacontainer/ISDWeather'

if dsetdata not in ws.datasets:
    os.system('sudo chmod 777 /mnt')
    for year in range(2008, 2020+1):
        ds = Dataset.File.from_files(f'{data_url}/year={year}/month=*/*.parquet', validate=False)
        print('Downloading...')
        %time ds.download(f'/mnt/data/isd/year={year}', overwrite=True)
    print('Uploading...')
    %time ws.get_default_datastore().upload('/mnt/data/isd', '/noaa-isd', show_progress=False)
    ds = Dataset.File.from_files((ws.get_default_datastore(), '/noaa-isd/**/*.parquet'))
    ds = ds.register(ws, dsetdata)

In [5]:
dsetdata = 'noaa-isd-tabular'

if dsetdata not in ws.datasets:
    ds = Dataset.Tabular.from_parquet_files((ws.get_default_datastore(), '/noaa-isd/**/*.parquet'))
    ds = ds.register(ws, dsetdata)

## Start Spark Session

In [6]:
# use any Synapse workspace/Spark pool you can access with AAD 
%spark start --workspace sparky --sparkpool sparky

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BQSA4WNQL to authenticate.
Starting Spark application ...
SparkSession available as 'spark'.


## Configure Spark Session

In [7]:
%%spark config
{
    "driverMemory":"128g",
    "driverCores":16,
    "executorMemory":"8g",
    "executorCores":1,
    "numExecutors":200
}

## Work with data

In [8]:
%%spark

# get workspace
from azureml.core import Workspace
subscription_id = '6560575d-fa06-4e7d-95fb-f962e74efd7a'
resource_group = 'cody-westus2-rg'
workspace_name = 'AzureML'

ws = Workspace(subscription_id, resource_group, workspace_name)
ws

Workspace.create(name='AzureML', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='cody-westus2-rg')

In [13]:
%%spark 

ds = ws.datasets['noaa-isd-tabular']
ds

{
  "source": [
    "('workspaceblobstore', 'noaa-isd/**/*.parquet')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "02c60e72-49c6-482c-9672-2dac6c5d7439",
    "name": "noaa-isd-tabular",
    "version": 1,
    "workspace": "Workspace.create(name='AzureML', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='cody-westus2-rg')"
  }
}

In [14]:
%%spark

df = ds.to_spark_dataframe()

An error was encountered:
An error occurred while calling o135.getFiles.
: java.lang.RuntimeException: Unable to get service host.
Workspace ID: 
Workspace ID Program Error: Could not find valid SPARK_HOME while searching ['/var/yarn-nm/tmp/nm-local-dir/usercache/trusted-service-user/appcache/application_1584502209143_0002', '/usr/hdp/2.6.99.201-0/spark2/python/lib/pyspark.zip/pyspark', '/opt/spark/python/lib/pyspark.zip/pyspark', '/opt/spark/python/lib']
Service Url Program Error: Could not find valid SPARK_HOME while searching ['/var/yarn-nm/tmp/nm-local-dir/usercache/trusted-service-user/appcache/application_1584502209143_0002', '/usr/hdp/2.6.99.201-0/spark2/python/lib/pyspark.zip/pyspark', '/opt/spark/python/lib/pyspark.zip/pyspark', '/opt/spark/python/lib']
	at com.microsoft.dprep.integration.azureml.ServiceDiscovery$.getHistoryUri(ServiceDiscovery.scala:37)
	at com.microsoft.dprep.integration.azureml.DatastoreServiceResolver.resolve(DatastoreResolver.scala:47)
	at com.microsoft.d

In [ ]:
%%spark

sc

### Show Session Details

In [ ]:
%spark info

### Stop Session

In [ ]:
%spark stop

### Override Session Config

In [ ]:
%%spark config
{
    "driverMemory":"8g",
    "driverCores":2,
    "executorMemory":"8g",
    "executorCores":2,
    "numExecutors":2
}

### Example: RDD operation

In [ ]:
%%spark

import time
b=sc.broadcast([3,5]) #Creating a broadcast variable available on all executors
a=sc.accumulator(0)   #Creating an accumulator for adding values across executors
RDD0=sc.parallelize([y for y in range(0,5)]) #RDD from input python collection
RDD2=sc.parallelize([z for z in range(10,15)])
RDD1=RDD0.cartesian(RDD2) 
cached=RDD2.cache() #Testing cached RDD
RDD22=RDD1.map(lambda x:x[0]+x[1]+b.value[0])
RDD3=RDD22.repartition(5) # To trigger a new stage.
RDD4=RDD2.map(lambda x: 3*x-b.value[0])
RDD5=RDD3.filter(lambda x:x%2==0)
RDD6=RDD4.filter(lambda x:x%2!=0)
RDD7=RDD5.cartesian(RDD6)
RDD8=RDD7.flatMap(lambda x: [x[i] for i in range(0,2)])
RDD9=RDD8.union(cached)
ans=RDD9.reduce(lambda x,y: x+y) # Doing a simple sum on the random data.
print(ans)
def f(x):
    global a
    time.sleep(0.7) #Making the job run a little longer
    a+=x
RDD9.foreach(f)
print(a.value)

### Example: Spark Pi

In [ ]:
%%spark

from random import random
from operator import add

partitions = 10
n = 100000 * partitions

def f(_):
    x = random() * 2 - 1
    y = random() * 2 - 1
    return 1 if x ** 2 + y ** 2 <= 1 else 0

count = sc.parallelize(range(1, n + 1), partitions).map(f).reduce(add)
print("Pi is roughly %f" % (4.0 * count / n))